# Running TriTorch

---


# ESC "OFF"

**Create racecar class**

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

#______________Reset car_____________
car.steering = -0.22  #-0.22 center
car.throttle = 0.14  #0.14  neutral

# ESC "ON"

**Create camera class**

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=15)

**Load model**

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('models/apexcolab40_trt.pth'))

**Run model**

In [ ]:
from utils import preprocess
import numpy as np
import os
import cv2
import time
import datetime
from IPython.display import clear_output
out=0


#____________Calibration____________
STEERING_BIAS = -0.22  #-0.22 center
THROTTLE_BIAS =  0.14  #0.14  neutral
#___________________________________


#****************CONTROLS*****************
STEERING_GAIN = -1.0   # -1 for traxxas
THROTTLE_GAIN =  0.1   # 0.4 tent track
MAX           =  0.21   # Max constant speed value. 0.22 max tent track
#DAMP          =  0.5
#THR           =  0.5 #threshold of steering
#****************OPTIONS******************
LIMITER       =  True # default MIN = 0.16
CONSTANT      =  False # car.throttle = MAX
#DAMPNER       =  False # car.throttle = MAX
PRINT         =  True  # outputs steering & throttle values
#*****************************************


try:
    while True:
    
        #______________Image processing_____________
        imageSeen = camera.read()
        image = preprocess(imageSeen).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        y = float(output[1])
        #___________________________________________

    
    
        #______________Steering control_____________
        STEERING = round(x * STEERING_GAIN + STEERING_BIAS, 2)  
        if STEERING < -1.0: #Servo motor protection
            car.steering = -1.0
        elif STEERING > 1.0:
            car.steering = 1.0
        #elif DAMPNER:
         #   if STEERING < THR and STEERING > -1*THR:
          #      car.steering = STEERING*DAMP
           # else: car.steering = STEERING
        else: car.steering = STEERING
        #___________________________________________

    
    
        #______________Throttle control_____________
        THROTTLE = round(y * THROTTLE_GAIN + THROTTLE_BIAS, 2)
        if LIMITER: #Limiter
            if THROTTLE < 0.16: #minimum speed
                car.throttle = 0.16
            elif THROTTLE > MAX: #maximum speed
                car.throttle = MAX
            else: car.throttle = THROTTLE
        elif CONSTANT:
            car.throttle = MAX
        else: car.throttle = THROTTLE
        #___________________________________________

        
        #________________Print value________________
        if PRINT: #output value
            print("S = "+str(car.steering)+" T = "+str(car.throttle))
            if out == 10:
                clear_output()
                out = 0
            out+=1
        #___________________________________________
        
        
except KeyboardInterrupt:
#______________Reset car_____________
    car.steering = -0.22
    car.throttle = 0.14
    print ("!!!!!!!!STOP!!!!!!!!")
#____________________________________

In [ ]:
#______________Reset car_____________
car.steering = -0.22
car.throttle = 0.14